In [83]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.stem import WordNetLemmatizer
from nltk import regexp_tokenize
from nltk.corpus import gutenberg 
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity
import random

In [38]:
nltk.download('stopwords')
nltk.download('punkt') 
nltk.download('gutenberg')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [95]:
#Get all books
book1 = nltk.corpus.gutenberg.raw('edgeworth-parents.txt')
book2 = nltk.corpus.gutenberg.raw('melville-moby_dick.txt')
book3 = nltk.corpus.gutenberg.raw('carroll-alice.txt')
book4 = nltk.corpus.gutenberg.raw('chesterton-brown.txt')
book5 = nltk.corpus.gutenberg.raw('milton-paradise.txt')

books=[book1,book2,book3,book4,book5]
names=[]

#Get Labels
for book in books:
  add_names=re.findall("(\[[ a-zA-Z0-9 '_.+-|\]+ by [ a-zA-Z0-9-]+|\.[ a-zA-Z0-9-.]+\])", book)
  names.append(''.join(add_names[0]))

split_names = []
for a_name in names: 
  name = a_name.partition(" by ")[2].partition(']')[0]
  name = ''.join([i for i in name if not i.isdigit()])

  #split=re.split('by',a_name)    
  split_names.append(name)
  #print(split_names)

labels = []
#for new in split_names:
#  split_a = new[0]
 # split_b = re.sub(r'[\d+\\!"#$%&()*+,-./:;?@[\]^_`{|}~]','',split_a)
  #labels.append(split_b)

d = dict(zip(books, split_names))
df_final = pd.DataFrame(columns=['Text', 'Labels'])
df = pd.DataFrame(columns=['Text', 'Labels'])

  #Partition Data  
for text in books:
  doc = []
  final_text = clean_text(text).split()
  for i in range(0, 100):
    for j in range(0, 100):
      data_1 = (random.sample(final_text, j))
    doc.append(' '.join(str(''.join(str(x) for x in v)) for v in data_1))
  df['Text'] = doc
  df['Labels'] = d[str(text)]
  df_final = df_final.append(df, ignore_index=True)
       
       
print(df_final)

                                                  Text           Labels
0    indeed song let fatigue showy lest said though...  Maria Edgeworth
1    quarrelling wistful done said money great box ...  Maria Edgeworth
2    father isabella place uncle ask acquired added...  Maria Edgeworth
3    seem following go landlady dear two settle u s...  Maria Edgeworth
4    look example father given one nigh said perhap...  Maria Edgeworth
..                                                 ...              ...
495  grace broke foundation heaven plutonian power ...     John Milton 
496  land fly liquours groaned infinite yet well pl...     John Milton 
497  love yet hard soft wood unsafe another ark th ...     John Milton 
498  simply every taste dwell flower oft bad one ho...     John Milton 
499  distance lucifer rolled port blazing design az...     John Milton 

[500 rows x 2 columns]


In [96]:
sentences = []
for s in df_final['Text']:
  sentences.append(sent_tokenize(s))

sentences = [y for x in sentences for y in x]

In [97]:
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [98]:
sentences[:5]

['indeed song let fatigue showy lest said thought stooping starting plump regretted prisoner declare piece pay girl watching write brat even say several mr garret matter father sure sooner show heartily night day tomorrow want skirt matter always observation said tease tell turned boy guilty laura hal lest money plait enough attorney shop lose cannot frightened money immediately temper barring talbot thought selfish well man told though friend said liked francisco matter put upon try said virtue hope genius recollected run shilling boy boy one susan pointing somerville indeed eye francisco evening people may wicked would still father praising',
 'quarrelling wistful done said money great box friendship loved vicarage depended inquiry submitted day smiled let also black rosamond father admiration arrow first anything ale flannel given whole hand awhile caught work done said ought time cecilia soldier lamb candle nothing booty neighbour reader afraid must look ye letting laura half chanc

In [63]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip '/content/glove*.zip'

--2021-02-06 14:47:09--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-02-06 14:47:09--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-02-06 14:47:09--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [99]:
word_embeddings = {}
f = open('/content/glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [100]:
len(word_embeddings)

400000

In [101]:
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

clean_sentences = [s.lower() for s in clean_sentences]

In [102]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

In [103]:
sim_mat = np.zeros([len(sentences), len(sentences)])

In [104]:
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [105]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [106]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [107]:
for i in range(10):
  print(ranked_sentences[i][1])

poison longer know mouse hole piece think try sometimes rather back caterpillar vanished round game must like pointed thought soon march kick politely boot dear never always getting idea tied one get tell stood opening sitting eagerly bleeds help one voice taught beautify much table duchess rate tell one found face never station small creature sound certain soothing added called far thing put one darkness tossing anything journey adventure little voice hot opportunity x hurried life mean queen missed question name pale quite replied everything jury began little pigeon late gryphon top remembered seemed king said away day sage
blue stronger enough sort touch usual next betook field paul typical fresh shall happened life open portrait come passage remarkable word restraint towards make waiting felt hat half quarrel ended short consumed ungovernable said everything spear instant walking ever another sky could impossibility end struck college like great priest face man corpse major break a